In [9]:
import ujson

In [10]:
filename = 'annotations.json' ## insert filename here
with open(filename) as f:
    data = ujson.load(f)

TRAINING_DATA = []

In [11]:
for d in data:
    tuple_1 = d['data']['response']
    tuple_2 = {}
    entities = []
    results = d['annotations'][0]['result']
    for x in results:
        entities.append((x['value']['start'],  x['value']['end'], x['value']['labels'][0]))
    tuple_2['entities'] = entities
    TRAINING_DATA.append((tuple_1, tuple_2))

In [13]:
from spacy.tokens import DocBin
import spacy
db = DocBin()
nlp = spacy.blank("en")

bad = 0

for text, annotations in TRAINING_DATA:
    doc = nlp(text)
    ents = []
    for start, end, label in annotations['entities']:
        # print(text, text[start:end], label)
        span = doc.char_span(start, end, label=label)
        ents.append(span)
    try:
        doc.ents = ents
        db.add(doc)
    except:
        bad += 1
db.to_disk("./train.spacy")
print(bad)


10
